## **Create dataset for Time-dependent Deep Image Prior (TD-DIP) using 1.5T data**
This code takes the data from the MC-CINE at 1.5T, an create a new dataset wit the used format for the TD-DIP acquitecture

#### Libraries

In [1]:
# basic libraries
import numpy as onp
import jax.numpy as np 
import matplotlib.pyplot as plt  
import os
# bart
from inrmri.bart import bart_acquisition_from_arrays

In [2]:
from inrmri.utils_rdls import preprocess_data, get_csm_format
from inrmri.utils_rdls import plot_angles_and_times_from_xdata, balance_spokes_duplicating
from inrmri.utils_rdls import plot_traj, plot_kdata, plot_csm, plot_zoom_images

In [3]:
from inrmri.utils_rdls import save_frames_as_gif_with_pillow

# **1 .- DATA**

#### **Parameters**

In [4]:
# input folders
base_path             = '/mnt/workspace/datasets/CRUZ_TOM_FREERUN/CINE_tr_resolved/Processed_Data/CINE/'
slice_num             = 1
nslices               = 8
volunteer_name        = 'DW' # 'OJ', 'NF' 'GC', 'CM', 'DW'
# parameters
nbins                 = 30
# computation folders
bart_files_folder     = '/mnt/workspace/datasets/pulseqCINE/bart_files/'
# output folder
dataset_name          = 'DATA_1.5T'
base_folder           = '/mnt/workspace/datasets/pulseqCINE/' + dataset_name + '/' + volunteer_name + '/'
train_data_folder     = base_folder + '/traindata/'
recons_folder         = base_folder + '/recons/' 

In [5]:
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

if not os.path.exists(train_data_folder):
    os.makedirs(train_data_folder)

if not os.path.exists(recons_folder):
    os.makedirs(recons_folder)

In [6]:
dataset_name              = 'slice_' + str(slice_num) +'_' + str(nslices) +'_nbins' + str(nbins)
dataset_recon_folder      = recons_folder + dataset_name + '/'
path_save                 = train_data_folder + dataset_name + '.npz'

## **1.1 Load Data**

#### For each volunteer the following data is available: 
* **'traj'**:  Positions of the sampled data in k-space.
* **'dcf'**:   Density compensation.
* **'kdata'**: Acquired data.
* **'csm'**:   Sensitivity coil maps.
* **'Nt'**:    Total de spokes adquiridas.
* **'zf'**:    Reconstrucción con zero fill; supongo que juntando todas las spokes.
* **'fs'**:    Fully sampled recontruction.

In [7]:
zf     = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'zf' + '.npy', allow_pickle=True)[:,:,0,:]
traj   = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'traj' + '.npy', allow_pickle=True)[0]
kdata  = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'kdata' + '.npy', allow_pickle=True)[0]
fs     = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'fs' + '.npy', allow_pickle=True)[:,:,0,:]
dcf    = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'dcf' + '.npy', allow_pickle=True)[0]
csm    = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'csm' + '.npy', allow_pickle=True)[:,:,0,:]

# ---   data preprocess   --- 
# get information
readout, _, n_coils  = csm.shape
n_frames             = len(traj)
print('readout: ', readout)
print('number of coils:  ', n_coils)
print('number of frames: ', n_frames)

readout:  256
number of coils:   8
number of frames:  30


## **1.2 Preprocess data**

### Trajectory

In [8]:
# The trajectory is normalized, so we ponderate it by the readout
for i in range(nbins):
    traj[i] = traj[i]*readout

traj_aux_list = []
spokes_per_bin_list = []
bins_sorted = []
for i in range(n_frames):
    spokes_per_bin = traj[i].shape[0]//readout
    spokes_per_bin_list.append(spokes_per_bin)
    bins_sorted += [i]*spokes_per_bin
    traj_aux_list.append( np.reshape( traj[i], (spokes_per_bin, readout, 2) ) )

traj_sorted = np.concat(traj_aux_list, axis=0)
traj_sorted.shape

2025-05-02 18:22:22.641752: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version (12.6.20). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


(448, 256, 2)

### Coil sensitivities

In [9]:
# transpose coils
csm = np.transpose(csm, (2, 0, 1))
csm.shape

Exception ignored in: <function _xla_gc_callback at 0x7fa7ac67d3a0>
Traceback (most recent call last):
  File "/mnt/workspace/rdelasotta/.conda/envs/jaxenv/lib/python3.11/site-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


KeyboardInterrupt: 

### DATA

In [ ]:
kdata_aux_list = []
for i in range(n_frames):
    kdata_aux_list.append( kdata[i] )

In [ ]:
kdata_sorted = np.concat(kdata_aux_list, axis=1)
kdata_sorted.shape

## **1.2 Check data**

### **Trajectory (traj)**
##### The trajectory is a list of frames. The frames dimensions are (n_spokes*readout, 2)

In [ ]:
plot_traj(traj_sorted, spokes_per_bin_list, vplot = 2, hplot = 4)

### **k-space data (kdata)**
##### For each point of the trajectory, a signal was acquired by each coil. The data corresponds to the horizontal magnetization, obtained by a pulse sequence. Given that we have two axis in the horizontal plane, the data is in complex form.
##### The kdata is a list of frames. The frames dimensions are (readout, n_spokes, coils)

In [ ]:
plot_kdata(kdata_sorted, spokes_per_bin_list, vplot = 2, hplot = 4)

### **Coil Sensitivity Map (CSM)**

In [ ]:
plot_csm(csm, spokes_per_bin_list, vplot = 2, hplot = 4)

# **2 .- TD-DIP dataset**

### times

In [ ]:
times = onp.array(bins_sorted)/n_frames

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 4))
axes.plot(times)
axes.set_title("Times")
plt.show()

## **2.3 .- Preprocess data**

In [ ]:
Y_data, X_data, spclim = preprocess_data(times, kdata_sorted, traj_sorted)
Y_data.shape, X_data.shape

In [ ]:
plot_angles_and_times_from_xdata(X_data)

# **3 .- GRASP and iterative-SENSE reconstructions**

## **3.1 .- Coil sensitivity map**

In [ ]:
traj_sorted.shape, kdata_sorted.shape

In [ ]:
csm_computed, hollow_mask_computed, bac_csm = get_csm_format(traj_sorted, kdata_sorted, bart_files_folder, dataset_name)
hollow_mask_computed = np.nan_to_num(hollow_mask_computed, nan=1.0)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

axes[0].imshow(onp.abs(csm_computed[0,:,:]))
axes[0].axis('off')  # Turn off the axis for the first image

axes[1].imshow(onp.abs(hollow_mask_computed))
axes[1].axis('off')  # Turn off the axis for the second image

plt.show()

## **3.2 .- GRASP reconstruction**

In [ ]:
ktraj_b, kdata_b, time_b = balance_spokes_duplicating(times, kdata_sorted, traj_sorted)

In [ ]:
bac_name_grasp = dataset_name + '_grasp_b'
bac_grasp = bart_acquisition_from_arrays(ktraj_b, kdata_b, bart_files_folder, bac_name_grasp)

In [ ]:
iters = 100
lagrangian_value = 5.
lambda_value = 0.01
grasp_exp_name = bac_name_grasp + str(iters) + '_lagrangian' + str(lagrangian_value).replace(".", "_") + '_lambda' + str(lambda_value).replace(".", "_")

recon_grasp = bac_grasp.calculate_bart_reco_with_external_csmap(bac_csm, grasp_exp_name, lmbda = lambda_value, lagrangian = lagrangian_value, iters = iters)

In [ ]:
recon_grasp = recon_grasp.reshape((recon_grasp.shape[0],recon_grasp.shape[1],recon_grasp.shape[-1]))
recon_grasp.shape

## **3.3 .- SENSE reconstruction**

In [ ]:
bac_name_sense = dataset_name + '_sense_b'
recon_sense = bac_grasp.make_sense_reconstruction( bac_csm.csmappath(), bac_name_sense)

In [ ]:
recon_sense = recon_sense.reshape((recon_sense.shape[0],recon_sense.shape[1],recon_sense.shape[-1]))
recon_sense.shape

## **3.4 .- Plot**

In [ ]:
array_images = [fs,recon_grasp,recon_sense,zf]
array_names = ['ground truth', 'GRASP', 'SENSE', 'Zero fill']
plot_zoom_images(array_images, array_names, frame_to_show=0, iterative_zoom=4)

## **5 .- Save data**
### Train DATA
* **Y_data**:                (spokes, coils, readout,1)
* **X_data**:                (spokes, 2)
* **csm**:                   (coils, readout, readout) # to do 
* **csm_computed**:          (coils, readout, readout)
* **hollow_mask_computed**:  (readout, readout)
* **spclim**:                0.5
### Reconstructions
* **recon_fs**:              (readout, readout, frames)
* **recon_grasp**:           (readout, readout, frames)
* **recon_sense**:           (readout, readout, frames)
* **recon_zf**:              (readout, readout, frames)

In [ ]:
zf.shape, fs.shape

In [ ]:
onp.savez(path_save, Y_data=Y_data, X_data=X_data, csm=csm, csm_computed=csm_computed, hollow_mask_computed=hollow_mask_computed, spclim=spclim, recon_grasp=recon_grasp, recon_sense=recon_sense, recon_zf=zf, recon_fs=fs)

## 6 .- Save Recon 

In [ ]:
recon_grasp_cropped = recon_grasp[recon_grasp.shape[0]//4:3*recon_grasp.shape[0]//4,recon_grasp.shape[1]//4:3*recon_grasp.shape[1]//4,:]
recon_sense_cropped = recon_sense[recon_sense.shape[0]//4:3*recon_sense.shape[0]//4,recon_sense.shape[1]//4:3*recon_sense.shape[1]//4,:]
zf_cropped = zf[zf.shape[0]//4:3*zf.shape[0]//4,zf.shape[1]//4:3*zf.shape[1]//4,:]
fs_cropped = fs[fs.shape[0]//4:3*fs.shape[0]//4,fs.shape[1]//4:3*fs.shape[1]//4,:]

In [ ]:
save_frames_as_gif_with_pillow(dataset_recon_folder, recon_grasp_cropped / onp.max(onp.abs(recon_grasp_cropped)), filename='GRASP', vmax=1, saturation=0.8, fps=30)
save_frames_as_gif_with_pillow(dataset_recon_folder, recon_sense_cropped / onp.max(onp.abs(recon_sense_cropped)), filename='SENSE', vmax=1, saturation=0.8, fps=30)
save_frames_as_gif_with_pillow(dataset_recon_folder, zf_cropped / onp.max(onp.abs(zf_cropped)), filename='ZF', vmax=1, saturation=0.8, fps=30)
save_frames_as_gif_with_pillow(dataset_recon_folder, fs_cropped / onp.max(onp.abs(fs_cropped)), filename='FS', vmax=1, saturation=0.8, fps=30)